In [4]:
import numpy as np
import folium

In [12]:
import numpy as np
import folium
from folium.plugins import AntPath

# ---------------------------------------------------------
# 1. DC power flow on a Texas toy grid
# ---------------------------------------------------------

buses = ["Dallas", "Houston", "Austin", "San Antonio"]
slack_bus = "Dallas"

# Line data: (from, to, reactance)
lines = [
    ("Dallas", "Austin", 0.12),
    ("Dallas", "Houston", 0.20),
    ("Austin", "Houston", 0.10),
    ("Austin", "San Antonio", 0.08),
    ("Houston", "San Antonio", 0.25),
]

# Net injections in MW (generation minus load)
P_dict = {
    "Dallas":      400.0,
    "Houston":    -250.0,
    "Austin":     -100.0,
    "San Antonio": -50.0,
}

# Lat lon coordinates for each city
bus_coords = {
    "Dallas":      (32.78, -96.80),
    "Houston":     (29.76, -95.37),
    "Austin":      (30.27, -97.74),
    "San Antonio": (29.42, -98.49),
}

# ---------------------------------------------------------
# 2. Build B matrix and solve DC power flow
# ---------------------------------------------------------

n = len(buses)
bus_idx = {b: i for i, b in enumerate(buses)}

B = np.zeros((n, n))

for i_bus, j_bus, x in lines:
    i = bus_idx[i_bus]
    j = bus_idx[j_bus]
    b_ij = 1.0 / x
    B[i, i] += b_ij
    B[j, j] += b_ij
    B[i, j] -= b_ij
    B[j, i] -= b_ij

P = np.array([P_dict[b] for b in buses], dtype=float)

slack_index = bus_idx[slack_bus]
mask = np.ones(n, dtype=bool)
mask[slack_index] = False

B_red = B[mask][:, mask]
P_red = P[mask]

theta_red = np.linalg.solve(B_red, P_red)
theta = np.zeros(n)
theta[mask] = theta_red
theta[slack_index] = 0.0

theta_dict = {b: theta[bus_idx[b]] for b in buses}

print("Bus angles in radians")
for b in buses:
    print(f"{b}: {theta_dict[b]:.4f}")

# Compute line flows F_ij = (theta_i - theta_j) / x_ij
line_flows = {}
for i_bus, j_bus, x in lines:
    ti = theta_dict[i_bus]
    tj = theta_dict[j_bus]
    F_ij = (ti - tj) / x
    line_flows[(i_bus, j_bus)] = F_ij
    line_flows[(j_bus, i_bus)] = -F_ij

print("\nLine flows in MW")
for (i_bus, j_bus, x) in lines:
    F = line_flows[(i_bus, j_bus)]
    print(f"{i_bus} -> {j_bus}: {F:.2f} MW")

# ---------------------------------------------------------
# 3. Create interactive map centered over Texas
# ---------------------------------------------------------

map_center_lat = 31.0
map_center_lon = -97.0

m = folium.Map(
    location=[map_center_lat, map_center_lon],
    zoom_start=6,
    tiles="CartoDB positron",
)

# ---------------------------------------------------------
# 4. Add bus markers
# ---------------------------------------------------------

for b in buses:
    lat, lon = bus_coords[b]
    angle = theta_dict[b]
    inj = P_dict[b]
    popup_html = f"""
    <b>{b}</b><br>
    Angle: {angle:.4f} rad<br>
    Injection: {inj:.1f} MW
    """
    folium.CircleMarker(
        location=[lat, lon],
        radius=8,
        popup=popup_html,
        tooltip=b,
        color="red" if b == slack_bus else "blue",
        fill=True,
        fill_opacity=0.9,
    ).add_to(m)

# ---------------------------------------------------------
# 5. Animated transmission lines with correct direction
# ---------------------------------------------------------

flows = [abs(line_flows[(i_bus, j_bus)]) for (i_bus, j_bus, _) in lines]
max_flow = max(flows) if flows else 1.0

for (i_bus, j_bus, x) in lines:
    lat_i, lon_i = bus_coords[i_bus]
    lat_j, lon_j = bus_coords[j_bus]
    F_ij = line_flows[(i_bus, j_bus)]

    # Decide actual flow direction
    # If F_ij >= 0, flow goes i -> j
    # If F_ij < 0, flow goes j -> i
    if F_ij >= 0:
        path_coords = [[lat_i, lon_i], [lat_j, lon_j]]
        direction_label = f"{i_bus} → {j_bus}"
    else:
        path_coords = [[lat_j, lon_j], [lat_i, lon_i]]
        direction_label = f"{j_bus} → {i_bus}"

    width = 2 + 8 * abs(F_ij) / max_flow
    color = "green" if F_ij >= 0 else "orange"

    popup_html = f"""
    <b>{direction_label}</b><br>
    Flow magnitude: {abs(F_ij):.1f} MW<br>
    Reactance: {x}
    """

    # Static base line (for context)
    folium.PolyLine(
        locations=path_coords,
        weight=width,
        color=color,
        opacity=0.4,
        popup=popup_html,
        tooltip=direction_label,
    ).add_to(m)

    # Animated AntPath on top (this is the part that pulses and moves)
    AntPath(
        locations=path_coords,
        delay=600,              # smaller gives faster motion
        dash_array=[10, 20],
        weight=width,
        color=color,
        pulse_color="white",
        opacity=0.9,
    ).add_to(m)

# ---------------------------------------------------------
# 6. Save to HTML
# ---------------------------------------------------------

output_file = "texas_dc_powerflow.html"
m.save(output_file)
print(f"\nSaved interactive animated map to {output_file}")
print("Open that file in your browser to see all lines pulsing.")

Bus angles in radians
Dallas: 0.0000
Houston: -34.3376
Austin: -27.3974
San Antonio: -32.1102

Line flows in MW
Dallas -> Austin: 228.31 MW
Dallas -> Houston: 171.69 MW
Austin -> Houston: 69.40 MW
Austin -> San Antonio: 58.91 MW
Houston -> San Antonio: -8.91 MW

Saved interactive animated map to texas_dc_powerflow.html
Open that file in your browser to see all lines pulsing.
